<a href="https://colab.research.google.com/github/naoya1110/IP2_Lecture_2023/blob/main/2023_IP2_Step06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step06 データの読み込みと分析

## はじめに
Excel形式で保存されたデータを読み込み，グラフのプロットとデータの分析を行う方法について学習します。

## パッケージのインポート
データの読み込みにはPandas（パンダス）と呼ばれるパッケージを利用します。その他にもNumpyやMatplotlib，ファイル操作用のosも使用するのでインポートしておきます。


```
import pandas as pd                # Pandasのインポート
import numpy as np                 # Numpyのインポート
import matplotlib.pyplot as plt    # Matplotlibのインポート
import os                          # osのインポート
```



matplotlib単体ではグラフに日本語を表示することができません。以下のコードで`japanize-matplotlib`というパッケージのインストールとインポートを行うことで日本語を使えるようにします。



```
! pip install japanize_matplotlib    # japanize_matplotlibのインストール
import japanize_matplotlib           # japanize_matplotlibのインポート
```



## Google Driveのマウント
Google DriveをマウントしStep06のフォルダ（ディレクトリ）にアクセスできるようにします。左のファイルタブに`drive`というディレクトリが表示されればマウント完了です。


```
from google.colab import drive
drive.mount('/content/drive')
```



## Excelファイルの読み込み
左のファイルタブから`data_voltage_current.xlsx`のファイルを探し，パスをコピーします。コピーしたパスを`filepath`として使用します。Excelファイルの読み込みには`pd.read_excel()`という関数が便利です。`df`を実行するとデータを確認できます。`df.head()`はデータの先頭部分，`df.tail()`はデータの末尾部分を表示します。



```
filepath = "自分のExcelファイルのパスに変更"
df = pd.read_excel(filepath)    # Excelファイルをdfとして読み込み
df.head()                       # dfの先頭部分を表示
```



読み込んだデータ`df`はpandasの`DataFrame`というデータタイプになっています。


```
type(df)
```



## データのプロットとフィッティング
`data_voltage_current.xlsx`は実験で測定したデータであると仮定し，これをグラフにプロットしたり分析してみましょう。

`df`から電圧のデータのみを取り出すには次のように書きます。



```
voltage = df["電圧[V]"]    # dfから電圧のデータを取得
voltage.head()
```



同じように電流1のデータを取り出します。



```
current1 = df["電流1[A]"]   #dfから電流1のデータを取得
current1.head()
```



電圧と電流1の関係をプロットしてみましょう。



```
plt.rcParams["font.size"]=14    # グラフのフォントサイズを変更
plt.plot(voltage, current1, marker="o", linewidth=0, label="電流1")
plt.xlabel("電圧 [V]")
plt.ylabel("電流 [A]")
plt.legend()
plt.grid()
```



## フィッティング（近似）
プロットしたデータにはノイズが含まれているようですが電圧と電流1はおおよそ比例関係にあり，$y=ax+b$の式で近似（フィッティング）できそうです。この近似式において傾き$a$と切片$b$はパラメータです。このパラメータを求めるためには`np.polyfit(x_data, y_data, dim)`という関数を使うことができます。ここで`dim`は近似式の次数です。今回使う近似式は$y=ax+b$で一次式なので`dim=1`です。このように$x$と$y$のデータに対して近似式$y=f(x)$を求めることを[回帰分析](https://ja.wikipedia.org/wiki/%E5%9B%9E%E5%B8%B0%E5%88%86%E6%9E%90)と呼びます。



```
dim = 1    # 近似式の次数を指定
a, b = np.polyfit(voltage, current1, dim)    # フィッティングを実行
print(f"a = {a}")
print(f"b = {b}")
```



$a$と$b$の値が求まったので`current1`の近似値`current1_fit`を求めます。



```
current1_fit = a*voltage + b
```



実験値と近似値をプロットして比較してみましょう。また`plt.savefig()`を使って画像を保存しましょう。



```
plt.plot(voltage, current1, marker="o", linewidth=0, label="実験値")
plt.plot(voltage, current1_fit, label=f"近似線")
plt.xlabel("電圧 [V]")
plt.ylabel("電流 [A]")
plt.legend()
plt.grid()
plt.tight_layout()

plt.savefig("figure.png")    # 画像の保存
```



ところで，ここで求めた傾き$a$は電流と電圧の比例係数($I=aV$)なのでコンダクタンス$G$の値と同じです。したがって抵抗値$R$は$1/a$で求めることができます。このようにして未知の物体の抵抗値を求めることができます。



```
G = a
R = 1/G
print(f"抵抗値 = {R:.3f} [Ohm]")
print(f"コンダクタンス = {G:.3f} [S]")
```



## データの保存
電圧データ，実験値，近似値を統合して`df_save`というDataFrameを新しく作ります。



```
df_save = pd.DataFrame()    #新しいDataFrameを作成

df_save["電圧[V]"] = voltage           # df_saveにvoltageの列を追加
df_save["実験値[A]"] = current1        # df_saveにcurrent1の列を追加
df_save["近似値[A]"] = current1_fit    # df_saveにcurrent1_fitの列を追加

df_save.head()
```



df1_saveをExcelファイルとしてgoogleドライブに保存しましょう。



```
data_dir = "googleドライブ内のフォルダを指定"
filename = "current_data_analized.xlsx"
filepath = os.path.join(data_dir, filename)
df_save.to_excel(filepath, index=False)
```



## 練習問題1
電圧と電流2の実験値をプロットしてみましょう。

電流2の実験データをフィッティングしてみましょう。まずは何次式を使って近似すればよいか考え，`np.polyfit(x_data, y_data, dim)`を使ってパラメータ$a_1, a_2, ... , a_N, b$を求めましょう。求めたパラメータを使って近似値`current2_fit`を求め，実験値と合わせてグラフにプロットしてみましょう。

## 気象データのプロットと分析

気象庁のHPで公開化されいる[多度津の2021/5/3の気象データ](https://www.data.jma.go.jp/obd/stats/etrn/view/10min_s1.php?prec_no=72&block_no=47890&year=2021&month=5&day=3&view=)`data_tadotsu_20210503.xlsx`を`df_weather`として読み込みます。



```
filepath = "自分のExcelファイルのパスに変更"
df_weather = pd.read_excel(filepath)
df_weather.head()
```



この日の時刻と気温の関係をプロットしてみましょう。



```
minute = np.arange(1, 144, 1)*10    # 分単位の時刻データを作成
hour = minute/60    # 分単位の時間データから時間単位の時刻データを作成
temperature = df_weather["気温[C]"]

plt.plot(hour, temperature)
plt.xlim(0, 24)
plt.xticks(np.arange(0, 25, 2))
plt.ylim(10, 24)
plt.yticks(np.arange(10, 25, 2))
plt.xlabel("時刻")
plt.ylabel("気温[$^o$C]")
plt.grid()
```



この日の最高気温（`temperature`の最大値）を求めてみましょう。


```
temperature.max()
```



次の書き方でも同じことができます。


```
np.max(temperature)
```



同様に最低気温（`temperature`の最小値）を求めることができます。


```
temperature.min()
```





```
np.min(temperature)
```



平均気温（`temperature`の平均値）を求めてみましょう。


```
temperature.mean()
```





```
np.mean(temperature)
```



## 練習問題2
気圧や湿度，風速のデータについてもプロットし，それぞれの最大値，平均値，最小値を求めてみましょう。

## まとめ
Excelのデータを読み込んで分析する方法について学習しました。学んだことを使って実験レポートのグラフを作成してみましょう。